# Ryan Tschritter

## Research question/interests

**To find the pearson correlation coefficients for each statistic relative to goals scored in the current/next season.**

To find the coefficients I only need to follow the peason correlation coefficient formula and calculate the coefficient. This would mean calculating the means and sums of each statistic and then entering them into the formula.

**To find the largest correlation coefficient and plot that statistic against goals scored in the current/next season.**

From the results of the question above I would be able to pick the statistic with the highest correlation  and graph that statistic against goals scored in the current and next seasons.

In [2]:
# Milestone 2 Task 2 - Import the data into a dataframe using pandas
import pandas as pd
nhl2017 = pd.read_csv('../data/processed/Data301ProjectData2017.csv')
nhl2018 = pd.read_csv('../data/processed/Data301ProjectData2018.csv')

---
## Milestone 3 Task 1 - Exploratory Data Analysis

In [3]:
# Import Python Libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
# Import Data CSV Files
nhl2017 = pd.read_csv('../data/processed/clean2017.csv')
nhl2018 = pd.read_csv('../data/processed/clean2018.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/clean2017.csv'

In [5]:
# View the shape of the dataframes (ie. the number of columns and rows)
fstring = f"The shape of the 2017 dataframe is {nhl2017.shape} and the 2018 dataframe is {nhl2018.shape}."
print(fstring)

The shape of the 2017 dataframe is (250, 19) and the 2018 dataframe is (350, 19).


In [6]:
nhl2017.head()

,Rk,Name,GP,G,A,P,PIM,+/-,TOI,PPG,GWG,PPP,G/GP,A/GP,P/GP,SHOTS,SH%,HITS,BS
0,1,Connor McDavid,82,41,67,108,26,20,21:32,5,7,20,0.500,0.817,1.317,274,15.00%,28,46
1,2,Claude Giroux,82,34,68,102,20,28,20:21,9,1,36,0.415,0.829,1.244,193,17.60%,30,23
2,3,Nikita Kucherov,80,39,61,100,42,15,19:48,8,7,36,0.488,0.763,1.250,279,14.00%,31,15
3,4,Evgeni Malkin,78,42,56,98,87,16,18:59,14,7,38,0.538,0.718,1.256,239,17.60%,48,32
4,5,Nathan MacKinnon,74,39,58,97,55,11,19:53,12,12,32,0.527,0.784,1.311,284,13.70%,38,22


In [7]:
nhl2018.head()

,Rk,Name,GP,G,A,P,PIM,+/-,TOI,PPG,GWG,PPP,G/GP,A/GP,P/GP,SHOTS,SH%,HITS,BS
0,1,Nikita Kucherov,82,41,87,128,62,24,19:57,15,8,48,0.500,1.061,1.561,246,16.70%,44,31
1,2,Connor McDavid,78,41,75,116,20,3,22:50,9,9,33,0.526,0.962,1.487,240,17.10%,39,30
2,3,Patrick Kane,81,44,66,110,22,2,22:29,9,7,30,0.543,0.815,1.358,341,12.90%,21,18
3,4,Leon Draisaitl,82,50,55,105,52,2,22:35,16,5,29,0.610,0.671,1.280,231,21.70%,57,26
4,5,Brad Marchand,79,36,64,100,96,15,19:37,10,9,34,0.456,0.810,1.266,231,15.60%,46,19


## Column Legend of Dataframes

|Acronym|Meaning|
|---|---|
|Rk|Rank In List|
|GP|Games Played|
|G|Goals Scored|
|A|Assists|
|P|Points|
|Pim|Penalty Minutes|
|+/-|Plus Minus|
|TOI|Average Time on Ice|
|PPG|Power-Play Goals|
|GWG|Game-Winning Goals|
|PPP|Power-Play Points|
|G/GP|Goals Per Game Played|
|A/GP|Assists Per Game Played|
|P/GP|Points Per Game Played|
|SHOTS|Shots On Goal|
|SH%|Shot Percentage|
|HITS|Hits|
|BS|Blocked Shots|


In [8]:
# Compare to see if all columns are the same between both NHL dataframes.
print(all (nhl2017.columns == nhl2018.columns))

True


In [9]:
nhl2017.describe().T

,count,mean,std,min,25%,50%,75%,max
Rk,250.0,125.500000,72.312977,1.000,63.25000,125.500,187.750,250.000
GP,250.0,76.772000,6.986506,44.000,74.00000,80.000,82.000,82.000
G,250.0,19.832000,8.811034,3.000,14.00000,18.000,25.000,49.000
A,250.0,32.188000,12.380540,9.000,23.00000,29.000,39.000,68.000
P,250.0,52.020000,16.779805,32.000,38.00000,48.000,62.000,108.000
PIM,250.0,33.956000,20.689764,2.000,20.00000,30.000,43.000,187.000
+/-,250.0,2.272000,14.966026,-42.000,-7.00000,2.000,11.000,49.000
PPG,250.0,5.132000,3.745221,0.000,2.00000,4.500,7.000,20.000
GWG,250.0,3.200000,2.163962,0.000,2.00000,3.000,4.000,12.000
PPP,250.0,14.740000,8.727030,0.000,8.00000,13.000,20.000,42.000


In [10]:
nhl2018.describe().T

,count,mean,std,min,25%,50%,75%,max
Rk,350.0,175.500000,101.180532,1.000,88.25000,175.5000,262.75000,350.000
GP,350.0,74.888571,8.727103,33.000,71.00000,78.0000,82.00000,84.000
G,350.0,17.445714,9.779879,1.000,10.00000,15.0000,22.00000,51.000
A,350.0,28.060000,13.532275,6.000,19.00000,24.0000,35.00000,87.000
P,350.0,45.505714,20.115532,23.000,30.00000,39.0000,54.75000,128.000
PIM,350.0,33.388571,20.920449,4.000,20.00000,28.0000,42.00000,153.000
+/-,350.0,1.134286,13.583098,-41.000,-8.00000,0.0000,11.00000,39.000
PPG,350.0,3.797143,3.857266,0.000,1.00000,3.0000,6.00000,20.000
GWG,350.0,2.768571,1.968414,0.000,1.00000,2.5000,4.00000,10.000
PPP,350.0,11.145714,9.013452,0.000,5.00000,9.0000,16.00000,48.000


## Comments about the dataframe describes

- 